In [12]:
# We disallow the query that potential produces bad data.


In [1]:
import beaapi
import numpy as np
import pandas as pd

In [2]:
# Get key from unversioned file
import os
from dotenv import load_dotenv
load_dotenv()
beakey = os.environ.get("beakey") #and be careful as USERID is passed back in the 'param' dict of many return values.

# API error

In [9]:
# We get an error if we ask for two tables from Input Output

#If we ask for single tables everything is fine, and we can easily get the index for the table
# Note: Table 56 is IxC_TR_1997-2020_PRO_SEC (from xlsx zip)  and 57 is _SUM.
spec1a = {"Year":"2010","tableID":"56"}
spec1b = {"Year":"2010","tableID":"57"}
tbl1a = beaapi.get_data(beakey, "InputOutput", **spec1a)
tbl1b = beaapi.get_data(beakey, "InputOutput", **spec1b)
print(tbl1a.duplicated(['TableID', 'Year','RowCode','ColCode','ColDescr']).sum()==0)  # True
print(tbl1b.duplicated(['TableID', 'Year','RowCode','ColCode','ColDescr']).sum()==0)  # True

# But if we do a combined query ask for both of the previous tables together, we get duplicate rows
spec2 = {"Year":"2010","tableID":"56,57"} #Just add the two table together
tbl2 = beaapi.get_data(beakey, "InputOutput", **spec2)
print(tbl2.duplicated(['TableID', 'Year','RowCode','ColCode','ColDescr']).sum()==0)  # False!
print(tbl1a.shape[0], tbl1b.shape[0], tbl2.shape[0]) # Rows don't add up

# Looking at some example rows, we can see
display(tbl1a.query('TableID==56 and Year=="2010" and RowCode=="11" and ColCode=="22" and ColDescr=="Utilities"'))
# 0.0102950 is correct and the other is a bad value.
# If we tbl2.to_csv('local/temp.csv') then it looks like there are 
#   sets of 4 rows with the same DataValue and (ColCodes, ColDesc), 1 being right and 3 being wrong. 
#   Some of the wrong ColCodes seem to be from table 57.
display(tbl2.query('TableID==56 and Year=="2010" and RowCode=="11" and ColCode=="22" and ColDescr=="Utilities"')) # Just differ in terms of DataValue!


True
True
False
272 5256 10016


,TableID,Year,RowCode,RowDescr,RowType,ColCode,ColDescr,ColType,DataValue,NoteRef
19,56,2010,11,"Agriculture, forestry, fishing, and hunting",Industry,22,Utilities,Commodity,0.010295,56


,TableID,Year,RowCode,RowDescr,RowType,ColCode,ColDescr,ColType,DataValue,NoteRef
73,56,2010,11,"Agriculture, forestry, fishing, and hunting",Industry,22,Utilities,Commodity,0.005105,56
128,56,2010,11,"Agriculture, forestry, fishing, and hunting",Industry,22,Utilities,Commodity,0.010295,56
